In [1]:
import mmsdk
import os
import argparse
import re
import csv
from mmsdk import mmdatasdk
from os import listdir
from os.path import isfile, join

In [2]:
# args = ["All Labels.csd", "COVAREP.csd", "FACET 4.2", "glove_vectors.csd", "OpenFace_2.csd"]
args = "../final_aligned"
dataset_dictionary={}

if os.path.isdir(args) is False:
	print ("Folder does not exist ...")
	exit(-1)

csdfiles = [f for f in listdir(args) if isfile(join(args, f)) and f[-4:]=='.csd']
if len(csdfiles)==0:
	print("No csd files in the given folder")
	exit(-2)

print("%d csd files found"%len(csdfiles))
for csdfile in csdfiles:
	dataset_dictionary[csdfile]=os.path.join(args,csdfile)
dataset=mmdatasdk.mmdataset(dataset_dictionary)

print ("List of the computational sequences")
print (dataset.computational_sequences.keys())

5 csd files found
[2020-10-25 02:23:16.442] | Success | Computational sequence read from file ../final_aligned\All Labels.csd ...
  3%|▎         | 631/23240 [00:00<00:07, 3154.13 Computational Sequence Entries/s][2020-10-25 02:23:19.461] | Status  | Checking the integrity of the <All Labels> computational sequence ...
[2020-10-25 02:23:19.461] | Status  | Checking the format of the data in <All Labels> computational sequence ...
[2020-10-25 02:23:27.099] | Success | <All Labels> computational sequence data in correct format.
[2020-10-25 02:23:27.099] | Status  | Checking the format of the metadata in <All Labels> computational sequence ...
[2020-10-25 02:23:27.099] | Success | <All Labels> computational sequence metadata in correct format.
[2020-10-25 02:23:27.099] | Success | <All Labels> computational sequence is valid!
[2020-10-25 02:23:27.100] | Success | Computational sequence read from file ../final_aligned\COVAREP.csd ...
  2%|▏         | 417/23240 [00:00<00:10, 2100.32 Computat

In [3]:
# dimensions=["sentiment","happy","sad","anger","surprise","disgust","fear"]
# dataset["All Labels.csd"].data['r13W4H8ZhqI[0]']['features'][:] outputs the np.array(1,7) of the labels
# dataset["All Labels.csd"].data['r13W4H8ZhqI[0]']['features'].value works too
emotion_values = dataset["All Labels.csd"].data['-3g5yACwYnA[0]']['intervals'].value
print(emotion_values)

[[ 82.753 100.555]]


In [4]:
facetKeys = dataset['FACET 4.2.csd'].data['-3g5yACwYnA[0]']
print(facetKeys.keys())
facet = facetKeys["features"][:]
print(facet[0])
print(facet[-1])
# print(facet)
# Intervals are between [x - y] y is the time stamp that is related to the label. so [82.753 - 82.7667] means that 82.7667 is the key to 
# which csv row it relates to.

<KeysViewHDF5 ['features', 'intervals']>
[-1.54778  -0.249003 -0.459376 -1.82684  -1.45573   1.81828  -2.1252
 -1.49848  -1.00365  -1.28808  -0.687972 -0.543313 -0.801844 -0.91561
 -1.34823  -0.884152 -1.82109  -0.55071  -1.1744   -0.945148 -2.1373
 -1.86273  -1.68854  -0.556205 -1.40695  -1.49977  -0.507725 -1.15999
 -2.21403  -1.27079   0.999827  0.981707  2.22548  19.1914   -1.83387 ]
[-1.38435  -0.257029 -0.661961 -3.17185  -2.7336    2.00474  -1.39168
 -2.51091  -0.34616  -0.701625 -1.09845  -1.19124  -1.08136  -1.34108
 -1.62452  -1.09531  -1.90922  -1.62617  -1.57097  -0.780415 -1.79702
 -1.57759  -0.578868 -1.05967  -1.50606  -1.04452  -1.64504  -1.70312
 -2.75465  -1.82161   0.998988  0.98591   2.3685   13.9383   -1.09604 ]


In [5]:
# The time intervals start on frame 2842 and end on 3016. Being off by 1 frame is also okay, should not affect anything
s = emotion_values[0][0]// 0.033333467
e = emotion_values[0][1]// 0.033333467
print(s,e)

2482.0 3016.0


In [6]:
label_data = dataset["All Labels.csd"]
labels_keys = dataset["All Labels.csd"].keys()
headers = ["raw_name", "original_video", "segmented_video", "segment_number", "start", "end", "start_frame", "end_frame", "sentiment", "happy", "sad", "anger", "surprise", "disgust", "fear"]
seconds_per_frame = 0.033333467
with open("mosei_labels.csv", "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(headers)
    count = 0
    for label in labels_keys:
        start_end = label_data.data[label]['intervals'][:]
        emotions = label_data.data[label]['features'][:]
        # start = start_end[0][0] + .0333467
        start = start_end[0][0]
        end = start_end[0][1]
        start_frame = int(start // seconds_per_frame)
        end_frame = int(end // seconds_per_frame)
        video_name = re.sub(r"\[.*\]", "", label)
        segment = re.findall(r"\[(.*?)\]", label)[0]
        segmented_video = f"{video_name}_{segment}"
        list_to_write = [label, video_name, segmented_video, segment, start, end, start_frame, end_frame, *emotions[0]]
        # print(list_to_write)
        csvwriter.writerow(list_to_write)
print("## Done!")

## Done!


In [7]:
import pandas as pd
hf=pd.read_csv("mosei_labels.csv")
hf.head()

,raw_name,original_video,segmented_video,segment_number,start,end,start_frame,end_frame,sentiment,happy,sad,anger,surprise,disgust,fear
0,--qXJuDtHPw[0],--qXJuDtHPw,--qXJuDtHPw_0,0,23.199,30.325,695,909,1.000000,0.666667,0.000000,0.0,0.0,0.0,0.000000
1,-3g5yACwYnA[0],-3g5yACwYnA,-3g5yACwYnA_0,0,82.753,100.555,2482,3016,1.000000,0.666667,0.666667,0.0,0.0,0.0,0.666667
2,-3g5yACwYnA[1],-3g5yACwYnA,-3g5yACwYnA_1,1,119.919,125.299,3597,3758,0.666667,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,-3g5yACwYnA[2],-3g5yACwYnA,-3g5yACwYnA_2,2,4.840,14.052,145,421,0.000000,0.666667,0.666667,0.0,0.0,0.0,0.333333
4,-3g5yACwYnA[3],-3g5yACwYnA,-3g5yACwYnA_3,3,13.211,27.521,396,825,0.000000,0.333333,0.333333,0.0,0.0,0.0,0.000000
